<a href="https://colab.research.google.com/github/HiepPham1412/cifa_img_classification/blob/master/CIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Setup R kernel

As Google colab has not supported R as yet, we need to change the default python2 kernel to R kernel. This may take up to 5 minutes.

In [1]:
import datetime
t1 = datetime.datetime.now()
!wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x ./Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -p /conda  > /dev/null 2>&1 
!/conda/bin/conda install -c r r-rstan r-irkernel gxx_linux-64 -y -q > /dev/null 2>&1
!/conda/bin/R -e "IRkernel::installspec(name = 'python2', displayname = 'R', user = FALSE)"  > /dev/null 2>&1
!mkdir /root/.R/
!echo "CXX14FLAGS=-O3 -mtune=native -march=native -Wno-ignored-attributes -Wno-deprecated-declarations" > /root/.R/Makevars
t2 = datetime.datetime.now()
print(t2 - t1)

--2018-12-29 20:20:23--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.19.10, 104.16.18.10, 2606:4700::6810:130a, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.19.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62574861 (60M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh.1’

Miniconda3-latest-L 100%[===================>]  59.68M   102MB/s    in 0.6s    

2018-12-29 20:20:23 (102 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh.1’ saved [62574861/62574861]

mkdir: cannot create directory ‘/root/.R/’: File exists
0:00:57.099852


## Install necessary packages

Install necessary packages before training our model

In [3]:
!git clone https://github.com/HiepPham1412/cifa_img_classification.git
!/conda/bin/conda  install -c r r-dplyr 
!/conda/bin/conda install keras

fatal: destination path 'cifa_img_classification' already exists and is not an empty directory.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [6]:
!/conda/bin/conda install -c conda-forge/label/gcc7 r-keras 

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 



---



Now, go to menu **Runtime -> Manage Sessions -> TERMINATE**.  

Finally, **RECONNECT**.



---



## 1. Alexnet
Alexnet is the winning architecture in 2012 of ImageNet competition. Although its not the state of the art model, its simpler configuration is less computational expensive than the recent ones.

In this project, due to difference in input size, we will adjust 

In [0]:
library('tensorflow')

In [0]:
# loading the data
cifar <- dataset_cifar10()
# n_train <-9000
# n_test <-3000
# we only use 10% of total data set or 10,000 out of 60000
train_x <- cifar$train$x/255
train_y <- to_categorical(cifar$train$y, num_classes = 10)

# we prepare the TESTING dataset 
test_x <- cifar$test$x/ 255
test_y <- to_categorical(cifar$test$y, num_classes = 10) 

# verification of dimensions
dim(train_x) 
cat("Number of obs. in the training dataset:",
    dim(train_x)[[1]],
    "\nNumber of obs. in the testing dataset:",
    dim(test_x)[[1]])

# optimisation methos: ADAM, ie. Adaptive Momentum Estimation

opt <- optimizer_adam(lr = 0.0005, decay = 1e-6)

#-------model configuration
library('keras')

#Instantiate an empty model
AlexNet = keras_model_sequential()

AlexNet%>%  
    # 1st Convolutional Layer -------------------------------
  layer_conv_2d(filter = 96,
                kernel_size = c(5,5),
                padding = "same",         
                input_shape =c(32,32,3))%>%
    layer_activation("relu")%>%
    # Max Pooling [[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]
    layer_max_pooling_2d(pool_size=c(2,2), strides = c(2,2),padding = "same")%>%
    # 2nd Convolutional Layer -------------------------------
    layer_conv_2d(filter = 256,
                kernel_size = c(3, 3),
                strides=c(1,1),
                padding = "same") %>%  
    layer_activation("relu")%>%
    # Max Pooling [[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]
    layer_max_pooling_2d(pool_size=c(2,2), strides = c(2,2),padding = "same")%>%
    # 3rd Convolutional Layer -------------------------------
    layer_conv_2d(filter = 384,
                kernel_size = c(3, 3),
                strides=c(1,1),
                padding = "same")%>%
    layer_activation("relu")%>%
    # 4th Convolutional Layer -------------------------------
    layer_conv_2d(filter = 384,
                kernel_size = c(3, 3),
                strides=c(1,1),
                padding = "same") %>%  
    layer_activation("relu")%>%
    # 5th Convolutional Layer -------------------------------
    layer_conv_2d(filter = 265,
                kernel_size = c(3, 3),
                strides=c(1,1),
                padding = "same") %>%  
    layer_activation("relu")%>%
    # Max Pooling [[[[[[[[[]]]]]]]]]
    layer_max_pooling_2d(pool_size=c(2,2), strides = c(2,2),padding = "same")%>%
    # Passing it to a Fully Connected layer
    layer_flatten() %>%  
    # 1st Fully Connected Layer -----------------------------
    layer_dense(1024) %>%  
    layer_activation("relu") %>%  
    # Add Dropout to prevent overfitting
    layer_dropout(0.4)%>%
    # 2nd Fully Connected Layer ------------------------------
    layer_dense(1024)%>%
    layer_activation("relu")%>%
    # Add Dropout
    layer_dropout(0.4)%>%
    # 3rd Fully Connected Layer ------------------------------
    layer_dense(512)%>%
    layer_activation("relu")%>%
    # Add Dropout
    layer_dropout(0.4)%>%
    # Output Layer
    layer_dense(10)%>%
    layer_activation("softmax")

AlexNet %>%
  compile(loss = "categorical_crossentropy",
          optimizer = opt,
          metrics = "accuracy")

# we can print model summary and its architecture
summary(AlexNet)

# Evaluation of the model performance on the TESTING data
# before training
AlexNet %>% evaluate(test_x, test_y)


# TRAINING THE MODEL 
data_augmentation <- TRUE

if(!data_augmentation) {  
  history <-AlexNet %>% fit(train_x, train_y, batch_size = 32,
                epochs = 80,
                #validation_data = list(test_x, test_y),
                shuffle = TRUE,
                validation_split = 0.2)
} else {  
  # generating images
  gen_images <- image_data_generator(featurewise_center = TRUE,
                                     featurewise_std_normalization = TRUE,
                                     rotation_range = 20,
                                     width_shift_range = 0.30,
                                     height_shift_range = 0.30,
                                     horizontal_flip = TRUE)
  
  # Fit image data generator internal statistics to some sample data
  gen_images %>% fit_image_data_generator(train_x)
  
  # Generates batches of augmented/normalized data from image data and 
  # labels to visually see the generated images by the model
  history <- AlexNet %>% fit_generator(
    flow_images_from_data(train_x, 
                          train_y, 
                          gen_images,
                          batch_size = 32 #, save_to_dir="~/CNNcifarimages/"
    ),
    steps_per_epoch = as.integer(50000/32),
    epochs = 80,
    #validation_data = list(test_x, test_y),
    validation_split = 0.2)
}

# Evaluation of the model performance on the TESTING data
AlexNet %>% evaluate(test_x, test_y)

[1] 50000    32    32     3

Number of obs. in the training dataset: 50000 
Number of obs. in the testing dataset: 10000________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
conv2d_59 (Conv2D)                  (None, 32, 32, 96)              7296        
________________________________________________________________________________
activation_98 (Activation)          (None, 32, 32, 96)              0           
________________________________________________________________________________
max_pooling2d_35 (MaxPooling2D)     (None, 16, 16, 96)              0           
________________________________________________________________________________
conv2d_60 (Conv2D)                  (None, 16, 16, 256)             221440      
________________________________________________________________________________
activation_99 (Activation)          (None, 16, 16, 256)             0           
__________________

$loss
[1] 2.303778

$acc
[1] 0.1

In [37]:
plot(history)

ERROR: ignored

## Transfer-learning

Transfer-learning is a way to leverage the power of pre-trained models. In paritcular,  if there is a well-tuned model trained on a large and diversifided image data set, we expect that the convolutional layers of these models will learn common patterns from these images.

Concretely, we use the convolutional layers 


*   Xception
*   InceptionV3
*   ResNet50
*   VGG16
*   MobileNet








------



```
# This is formatted as code
```


Now, go to menu **Runtime -> Manage Sessions -> TERMINATE**.  

Finally, **RECONNECT**.